# Demo environment

This notebook will be used as an introduction to the control and image constructs available within the demo environment of both the robot interfaces and the camera interfaces.

We will cover the steps necessary to connect to and control the robot while also collecting image data from the cameras mounted around and on the robotic arm.

## Camera Access

In this demo we'll be utilizing Intel's [RealSense D435i](https://www.intelrealsense.com/depth-camera-d435i/) for use with out robotic arm. We'll not only want to view these images, but we'll want to collect them as we perform tasks, as well. There any many use cases and approaches one can take when peforming tasks using robotics arms such as these that span the areas of standard supervised, unsupervised, and reinforcement learning. For this reason there will likely be a few combinations of data collection that we may want to employ.

We will start with building the software constructs necessary to view and collect data from the Intel RealSense camera that is connected to the arm of our robot.

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
import os
from ipywidgets import Image, HBox
import PIL.Image
from matplotlib import pyplot as plt
import sys
import threading
import time
import math
import traitlets

# Camera pipelines
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 30)
config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 30)

pipeline.start(config)

image_widget = Image(format='jpeg')

def bgr8_to_jpeg(value, quality=75):
    return bytes(cv2.imencode('.jpg', value)[1])

def update_img():
    frames = pipeline.wait_for_frames()
    depth_frame = frames.get_depth_frame()
    color_frame = frames.get_color_frame()

    depth_image = np.asanyarray(depth_frame.get_data())
    color_image = np.asanyarray(color_frame.get_data())

    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.3), cv2.COLORMAP_JET)

    images = np.hstack((color_image, depth_colormap))
    image_widget.value = bgr8_to_jpeg(images)
    
def view_camera():
    running = True
    thread = threading.Thread(target=update_img())
    thread.start()

In [25]:
cam = view_camera()

display(image_widget)

Image(value=b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x00\x00\x01\x00\x01\x00\x00\xff\xdb\x00C\x00\x02\x01\x0…

## Robotic Arm Access

The arm that we are utilizing in this demo is the [ufactory xarm](https://www.xarm.cc/). The company is generous enough to provide an open source python SDK for development against the platform. We will be utilizing this SDK heavily in this demo. It is assumed that the controller being used (in our case an Ubuntu 18.04) already has this SDK installed. If it does not, please follow the installation instructions found [here](https://github.com/xArm-Developer/xArm-Python-SDK#installation)

In [14]:
from xarm.wrapper import XArmAPI

arm = XArmAPI('192.168.1.244')
arm.motion_enable(enable=True)
arm.set_mode(0)
arm.set_state(state=0)
speed = 50

main-socket connect 192.168.1.244 success
report-socket connect 192.168.1.244 success
is_old_protocol: False
version_number: 1.3.1
=============sync_all
[motion_enable], xArm is not ready to move
[set_state], xArm is ready to move


In [22]:
arm.set_servo_angle(angle=[0, 0, -20, -90, 0], speed=speed, wait=True)

0